In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''  ## to use CPU
import glob
import sys
from audioset import vggish_embeddings

import keras

/home/deepank/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


Using TensorFlow backend.


In [2]:
keras_model = "Data/Trained_model_LSTM_2.h5"  ## unzip the model

folder = "06_6-09-10"

wav_directory = "/media/deepank/086C617F6C616880/extra_space/data_collection/sounds/sounds/"+ folder+ "/chunks_2"

tfrecord_file = None


def predict_laugh(processed_embedding):
    model = keras.models.load_model(keras_model)
    return model.predict(processed_embedding)


In [3]:
audio_embedder = vggish_embeddings.VGGishEmbedder(tfrecord_file)

files = glob.glob(wav_directory+'/*.wav')
embeddings = [audio_embedder.convert_audio_to_embedding(f) for f in files]

max_len = np.max([e.shape[0] for e in embeddings])

embeddings = np.array([np.append(e, np.zeros([(max_len - e.shape[0]), 128], np.float32), axis=0) for e in embeddings])

scores = predict_laugh(embeddings)

new = np.argmax(scores, axis=1)
new_1 = np.argsort(scores, axis = 1)

##argsort: Returns the indices that would sort an array in ascending order.


classes = pd.read_csv('Data/class_labels_reqd_latest_5.csv',header = None)  ### the csv where the class names are stored
classes_name = list(classes[2])

list_predictions = []
list_predictions_2 = []
naming = []
first_class = []
first_score = []
second_class = []
second_score = []
third_class = []
third_score = []
four_class = []
four_score = []
five_class = []
five_score = []


for names, argmax, argsort, score in zip(files, new, new_1, scores):
    
    list_predictions.append(classes_name[argmax])
    list_predictions_2.append(classes_name[argsort[-2]])

    print(names.split('/')[-1] + '\\\\' + classes_name[argsort[-1]].split(",")[0] + "  prob. =>" +  str(score[argsort[-1]]) + "\\\\" + 
         classes_name[argsort[-2]].split(",")[0] + "  prob.2 =>" + str(score[argsort[-2]]))

    naming.append((names.split('/')[-1].strip(".wav")) + ".jpg")
    first_class.append(classes_name[argsort[-1]].split(",")[0])
    first_score.append(str(score[argsort[-1]]))
    second_class.append(classes_name[argsort[-2]].split(",")[0])
    second_score.append(str(score[argsort[-2]]))
    third_class.append(classes_name[argsort[-3]].split(",")[0])
    third_score.append(str(score[argsort[-3]]))
    four_class.append(classes_name[argsort[-4]].split(",")[0])
    four_score.append(str(score[argsort[-4]]))
    five_class.append(classes_name[argsort[-5]].split(",")[0])
    five_score.append(str(score[argsort[-5]]))

    
    
import collections
print(collections.Counter(list_predictions))
print(collections.Counter(list_predictions_2))

import pandas as pd

import numpy as np
sound_clips_predictions = pd.DataFrame(np.column_stack([naming,first_class,first_score,second_class,second_score,third_class,third_score,four_class,four_score,
                                                five_class,five_score]), 
                               columns=['naming','first_class','first_score','second_class','second_score','third_class','third_score','four_class',
                                        'four_score','five_class','five_score'])

sound_clips_predictions.to_csv("Data/predictions_" + folder + ".csv", index = None)


INFO:tensorflow:Restoring parameters from audioset/vggish_model.ckpt
2018_06_06_09_46_45_369_+0530.wav\\Birds  prob. =>0.93236625\\Crowd  prob.2 =>0.0386775
2018_06_06_09_46_47_076_+0530.wav\\Birds  prob. =>0.89444023\\Crowd  prob.2 =>0.052334066
2018_06_06_09_46_48_305_+0530.wav\\Birds  prob. =>0.9738022\\Crowd  prob.2 =>0.021167023
2018_06_06_09_46_49_472_+0530.wav\\Birds  prob. =>0.93786997\\Crowd  prob.2 =>0.020556098
2018_06_06_09_46_51_106_+0530.wav\\Birds  prob. =>0.97322005\\Silence  prob.2 =>0.011758439
2018_06_06_09_46_52_335_+0530.wav\\Birds  prob. =>0.96841764\\Vehicles_passing  prob.2 =>0.020865161
2018_06_06_09_46_53_553_+0530.wav\\Birds  prob. =>0.9851354\\Vehicles_passing  prob.2 =>0.007202244
2018_06_06_09_46_55_247_+0530.wav\\Birds  prob. =>0.9592607\\Vehicles_passing  prob.2 =>0.02449026
2018_06_06_09_46_56_429_+0530.wav\\Birds  prob. =>0.8307061\\Vehicles_passing  prob.2 =>0.052454446
2018_06_06_09_46_59_049_+0530.wav\\Birds  prob. =>0.62607485\\Vehicles_passing  pr